# Download and downscale ERA5 daily products using [TopoPyScale](https://topopyscale.readthedocs.io/en/latest/)

In [ ]:
import pandas as pd
from TopoPyScale import topoclass as tc
from matplotlib import pyplot as plt

In [ ]:
# ========= STEP 1 ==========
# Load Configuration
config_file = '/Volumes/LaCie/raineyaberle/Research/PhD/SnowMaL/study-sites/MCS/ERA5/config.yml'
mp = tc.Topoclass(config_file)

In [ ]:
# ======== STEP 2 ===========
# Compute parameters of the DEM (slope, aspect, sky view factor)
mp.compute_dem_param()
# mp.extract_topo_param()

In [ ]:
# ========= STEP 3 ==========
# compute solar geometry and horizon angles
mp.compute_solar_geometry()
mp.compute_horizon()

In [ ]:
# ========= STEP 4 ==========
# Perform the downscaling
mp.downscale_climate()

In [ ]:
import numpy as np
mp.search_optimum_number_of_clusters(cluster_range=np.array([10, 50]))

In [ ]:
# ========= STEP 5 ==========
# explore the downscaled dataset. For instance the temperature difference between each point and the first one
(mp.downscaled_pts.t-mp.downscaled_pts.t.isel(point_id=0)).plot()
plt.show()

In [ ]:
# ========= STEP 6 ==========
# Export output to desired format
mp.to_netcdf()